In [ ]:
shp = '/mnt/g/otol_exp2/guavsforest.gdb/osm/osm_green_areas'

fid = 'ffid'

evu_cls = '14100'
for_cls = '31000'

out = '/mnt/g/otol_exp2/results/osm_green_classi_exp1.shp'

In [ ]:
import numpy as np
from glass.rd.shp import shp_to_obj
from glass.wt.shp import df_to_shp

In [ ]:
gdf = shp_to_obj(shp)

In [ ]:
is_gua, is_for = [], []

for idx, row in gdf.iterrows():
    if row["garea"] > 80:
        # It is not
        is_gua.append(0)

    else:
        # Check if it is GUA
        if row["garea"] > 7:
            # We guess it is not
            if row['npnts_evu'] >= 5 or row['lenlines_evu'] >=1 or row['poly_evu'] >= 25:
                if row['npnts_for'] == 0 and row['lenlines_for'] == 0:
                    # It is
                    is_gua.append(1)
                else:
                    #if row['tcd_mean'] > 60:
                        # Is is not
                        #is_gua.append(0)
                    #else:
                        #if row['denspop'] >= 1250:
                            # It is:
                            #is_gua.append(1)

                        #else:
                            # We don't know
                            #is_gua.append(0)
                    # We don't know
                    is_gua.append(0)
            else:
                # It is not
                is_gua.append(0)

        else:
            if row['npnts_evu'] >= 5 or row['lenlines_evu'] >=1 or row['poly_evu'] >= 25:
                if row['npnts_for'] == 0 and row['lenlines_for'] == 0:
                    # It is
                    is_gua.append(1)

                else:
                    # We don't know 
                    is_gua.append(0)

            else:
                if row['npnts_for'] >= 1 or row['lenlines_for'] >= 1:
                    # It is not
                    is_gua.append(0)

                else:
                    if row['tcd_mean'] >= 60:
                        # It is not
                        is_gua.append(0)

                    else:
                        if row['denspop'] >= 1250:
                            is_gua.append(1)

                        else:
                            if row['dirbup'] >= 1 or row['aurbbup'] >= 1 or row['edievu_bup'] >= 1:
                                is_gua.append(1)

                            else:
                                is_gua.append(0)
    # Check if it is Forest
    if row['garea'] >= 80:
        is_for.append(1)

    elif row['garea'] >= 1 and row['garea'] < 80:
        if row['npnts_for'] >= 1 or row['lenlines_for'] >=1:
            if row['npnts_evu'] < 5 and not row['lenlines_evu'] and row['poly_evu'] < 25:
                is_for.append(1)

            else:
                if row['tcd_mean'] >= 60:
                    is_for.append(1)
                else:
                    # We dont' know
                    is_for.append(0)

        else:
            if row['npnts_evu'] >= 5 or row['lenlines_evu'] >=1 or row['poly_evu'] >= 25:
                # It is not
                is_for.append(0)

            else:
                if row['denspop'] > 19000:
                    is_for.append(0)

                else:
                    if row['edievu_bup'] >= 45:
                        is_for.append(0)

                    else:
                        if row['tcd_mean'] >= 50:
                            is_for.append(1)
                        else:
                            if row['denspop'] < 1250:
                                is_for.append(1)

                            else:
                                is_for.append(0)

    # Bellow zero
    else:
        if row['npnts_for'] >= 1 or row['lenlines_for'] >=1:
            if row['npnts_evu'] < 5 and not row['lenlines_evu'] and row['poly_evu'] < 25:
                is_for.append(1)

            else:
                if row['tcd_mean'] >= 50:
                    is_for.append(1)
                else:
                    # We don't know
                    is_for.append(0)

        else:
            if row['tcd_mean'] >= 60:
                is_for.append(1)
            else:
                is_for.append(0)

gdf['is_gua'] = is_gua
gdf['is_for'] = is_for

gdf['classi'] = np.where(
    (gdf.is_gua == 1) & (gdf.is_for == 0),
    evu_cls, '0'
)

gdf['classi'] = np.where(
    (gdf.is_gua == 0) & (gdf.is_for == 1),
    for_cls, gdf.classi
)

gdf['clsarea'] = np.where(
    gdf.garea >= 1, for_cls, evu_cls
)

gdf['classi'] = np.where(
    gdf.is_gua == gdf.is_for,
    gdf.clsarea, gdf.classi
)

In [ ]:
df_to_shp(gdf, out)